<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 4.2: Feature Selection

## Load & Explore Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Load Data

In [2]:
# Read CSV
wine_csv = '../Data/winequality_merged.csv'
df = pd.read_csv(wine_csv)
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red_wine
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


## Explore Data (Exploratory Data Analysis)

In [3]:
# ANSWER
abs(df.corr()['quality']).sort_values(ascending=False)

quality                 1.000000
alcohol                 0.444319
density                 0.305858
volatile acidity        0.265699
chlorides               0.200666
red_wine                0.119323
citric acid             0.085532
fixed acidity           0.076743
free sulfur dioxide     0.055463
total sulfur dioxide    0.041385
sulphates               0.038485
residual sugar          0.036980
pH                      0.019506
Name: quality, dtype: float64

## Set Target Variable

Create a target variable for wine quality.

In [4]:
# Target Variable
y = df['quality']

# Set Predictor Variables based on correlation

Create a predictor matrix with variables of your choice. State your reason.

In [5]:
# ANSWER
# df.corr()[target_variable].sort_values()
# We select those with corelation of at least 0.1, thereby we eliminated slightly more than half of the features.

print(abs(df.corr()['quality']).sort_values())
X = df[['alcohol','density','volatile acidity','chlorides','red_wine']]

pH                      0.019506
residual sugar          0.036980
sulphates               0.038485
total sulfur dioxide    0.041385
free sulfur dioxide     0.055463
fixed acidity           0.076743
citric acid             0.085532
red_wine                0.119323
chlorides               0.200666
volatile acidity        0.265699
density                 0.305858
alcohol                 0.444319
quality                 1.000000
Name: quality, dtype: float64


In [6]:
X

,alcohol,density,volatile acidity,chlorides,red_wine
0,9.4,0.99780,0.70,0.076,1
1,9.8,0.99680,0.88,0.098,1
2,9.8,0.99700,0.76,0.092,1
3,9.8,0.99800,0.28,0.075,1
4,9.4,0.99780,0.70,0.076,1
...,...,...,...,...,...
6492,11.2,0.99114,0.21,0.039,0
6493,9.6,0.99490,0.32,0.047,0
6494,9.4,0.99254,0.24,0.041,0
6495,12.8,0.98869,0.29,0.022,0


# Using Linear Regression model

Reference: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [8]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [9]:
# Create a model for Linear Regression
lr = LinearRegression()

# Fit the model with the Training data
lr.fit(X_train, y_train)

# Calculate the score (R^2 for Regression) for Training Data
lr_score_train = lr.score(X_train, y_train)
# Calculate the score (R^2 for Regression) for Testing Data
lr_score_test = lr.score(X_test, y_test)

print(f"lr_score_train = {lr_score_train:.2f} and lr_score_test = {lr_score_test:.2f}")

lr_score_train = 0.27 and lr_score_test = 0.26


# Forward Feature Selection

> Forward Selection: Forward selection is an iterative method in which we start with having no feature in the model. In each iteration, we keep adding the feature which best improves our model till an addition of a new variable does not improve the performance of the model.

Create a Regression model using Forward Feature Selection by looping over all the features adding one at a time until there are no improvements on the prediction metric ( R2  and  AdjustedR2  in this case).

### Overview of the code below

The external `while` loop goes forever until there are no improvements to the model, which is controlled by the flag `changed` (until is **not** changed).
The inner `for` loop goes over each of the features not yet included in the model and calculates the correlation coefficient. If any model improves on the previous best model then the records are updated.

### Code variables
- `included`: list of the features (predictors) that were included in the model; starts empty.
- `excluded`: list of features that have **not** been included in the model; starts as the full list of features.
- `best`: dictionary to keep record of the best model found at any stage; starts 'empty'.
- `model`: object of class LinearRegression, with default values for all parameters.

### Adjusted $R^2$ formula
$$Adjusted \; R^2 = 1 - { (1 - R^2) (n - 1)  \over n - k - 1 }$$

In [10]:
## Flag intermediate output
show_steps = True   # for testing/debugging

In [11]:
## Use Forward Feature Selection to pick a good model

# start with no predictors
included = []
# keep track of model and parameters
best = {'feature': '', 'r2': 0, 'a_r2': 0}
# create a model object to hold the modelling parameters
model = LinearRegression() # create a model for Linear Regression
# get the number of cases in the test data
n: int = X_test.shape[0]

while True:
    changed = False
    
    if show_steps:
        print('') 

    # list the features to be evaluated
    excluded = list(set(X.columns) - set(included))
    
    if show_steps:
        print('(Step) Excluded = %s' % ', '.join(excluded))  

    # for each remaining feature to be evaluated
    for new_column in excluded:
        
        if show_steps:
            print('(Step) Trying %s...' % new_column)
            print('(Step) - Features = %s' % ', '.join(included + [new_column]))
           
        # fit the model with the Training data
        fit = model.fit(X_train[included + [new_column]], y_train) # fit a model; consider which predictors should be included
        # calculate the score (R^2 for Regression)
        r2 = fit.score(X_test[included + [new_column]], y_test) # calculate the score
        # number of predictors in this model
        k = len(included) + 1
        # calculate the adjusted R^2
        adjusted_r2 = 1 - (((1- r2)*(n-1))/(n-k-1)) # calculate the Adjusted R^2

        if show_steps:
            print('(Step) - Adjusted R^2: This = %.3f; Best = %.3f' % 
                  (adjusted_r2, best['a_r2']))

        # if model improves
        if adjusted_r2 > best['a_r2']:
            # record new parameters
            best = {'feature': new_column, 'r2': r2, 'a_r2': adjusted_r2}
            # flag that found a better model
            changed = True
            if show_steps:
                print('(Step) - New Best!   : Feature = %s; R^2 = %.3f; Adjusted R^2 = %.3f' % 
                      (best['feature'], best['r2'], best['a_r2']))
    # END for

    # if found a better model after testing all remaining features
    if changed:
        # update control details
        included.append(best['feature'])
        excluded = list(set(excluded) - set(best['feature']))
        print('Added feature %-4s with R^2 = %.3f and adjusted R^2 = %.3f' % 
              (best['feature'], best['r2'], best['a_r2']))
    else:
        # terminate if no better model
        break

print('')
print('Resulting features:')
print(', '.join(included))


(Step) Excluded = red_wine, volatile acidity, density, alcohol, chlorides
(Step) Trying red_wine...
(Step) - Features = red_wine
(Step) - Adjusted R^2: This = 0.021; Best = 0.000
(Step) - New Best!   : Feature = red_wine; R^2 = 0.021; Adjusted R^2 = 0.021
(Step) Trying volatile acidity...
(Step) - Features = volatile acidity
(Step) - Adjusted R^2: This = 0.075; Best = 0.021
(Step) - New Best!   : Feature = volatile acidity; R^2 = 0.075; Adjusted R^2 = 0.075
(Step) Trying density...
(Step) - Features = density
(Step) - Adjusted R^2: This = 0.102; Best = 0.075
(Step) - New Best!   : Feature = density; R^2 = 0.103; Adjusted R^2 = 0.102
(Step) Trying alcohol...
(Step) - Features = alcohol
(Step) - Adjusted R^2: This = 0.181; Best = 0.102
(Step) - New Best!   : Feature = alcohol; R^2 = 0.182; Adjusted R^2 = 0.181
(Step) Trying chlorides...
(Step) - Features = chlorides
(Step) - Adjusted R^2: This = 0.053; Best = 0.181
Added feature alcohol with R^2 = 0.182 and adjusted R^2 = 0.181

(Step) 

# Cross validation

In [12]:
n

1300

In [13]:
# Answer
df.shape

(6497, 13)

In [18]:
from sklearn.model_selection import cross_val_score

model = LinearRegression()
cross_val_score(model, X, y, scoring="neg_mean_squared_error")

array([-0.46183542, -0.64504919, -0.61770402, -0.58607489, -0.51370411])